In [1]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask
import openpyxl

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [4]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    
    """Базовая функция для связи с сервером"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query(bookquery.leaders, entity_id)
        for result in results:
            entities = {}
            entities['leader'] = result['leader']['value'].split('/')[-1]
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['leader'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
    
def positions(entity_id, f = 0):
    """Получение ответа с сервера и формирование датафреймов"""
    
    res = []
    if f > 5:
        global bad_requests
        bad_requests.append(entity_id)
        print('Bad request ', entity_id)
        return None
    try:
        results= execute_query(bookquery.leaders_position, entity_id)
        for result in results:
            entities = {}
            entities['person'] = result['person']['value'].split('/')[-1]
            entities['personLabel'] = result['personLabel']['value']
            entities['position'] = result['position']['value'].split('/')[-1]
            entities['positionLabel'] = result['positionLabel']['value']
            try:
                entities['startTime'] = result['startTime']['value']
                entities['startTimeNode'] = result['startTimeNode']['value']
            except:
                entities['startTime'] = "не указано"
                entities['startTimeNode'] = "не указано"
            try:
                entities['endTime'] = result['endTime']['value']
                entities['endTimeNode'] = result['endTimeNode']['value']
            except:
                entities['endTime'] = "не указано"
                entities['endTimeNode'] = "не указано"
            res.append(entities)
        return pd.DataFrame(data = res, columns = ['person', 'personLabel' ,'position' ,'positionLabel' ,'startTime' ,'startTimeNode','endTime' , 'endTimeNode'])
    except Exception as e:
        print(e)
        time.sleep(5)
        print('Try again ', entity_id, f)
        return entities_sparqul(entity_id, f + 1)
    
def file_creation(dic):
    for ID,name in dic.items():
        new_df = pd.DataFrame()
        data = entities_sparqul(ID)
        for k,v in data.iterrows():
            new_df = pd.concat([new_df,positions(v.leader)])
        new_df['real_ruler'] = 'F'
        new_df['Delete'] = 'F'
        new_df.to_excel(f'Rulers/{name}.xlsx', sheet_name = name ,encoding='utf-8-sig' , index = False)

In [5]:
dic_id = {"Q15180" : 'СССР' ,"Q159":"Россия", "Q183":"Германия" , "Q30":"США"}
file_creation(dic_id)